In [1]:
from msibi import MSIBI, State, Pair, Bond, Angle
import gsd
import gsd.hoomd
import time

In [2]:
%%bash

for dir in "states" "rdfs" "potentials"
do
    if [ -d $dir ]
    then
        rm -r $dir
    fi
done

In [3]:
%%bash
ls

4.1kT-1.27den.gsd
5.5kT-1.38den.gsd
6.2kT-1.27den.gsd
peek_msibi.ipynb


In [4]:
cg_traj_files = [
    "4.1kT-1.27den.gsd",
    "5.5kT-1.38den.gsd",
    "6.2kT-1.27den.gsd"
]

In [5]:
opt = MSIBI(
    integrator="hoomd.md.integrate.nvt",
    integrator_kwargs={"tau": 0.1},
    dt=0.001,
    gsd_period=100,
    n_steps=1e4,
    max_frames=5,
    verbose=False
)

## Add States

In [6]:
opt.add_state(
    State(name="A", kT=4.1, traj_file=cg_traj_files[0], alpha=0.20)
)
opt.add_state(
    State(name="B", kT=5.5, traj_file=cg_traj_files[1], alpha=0.40)
)
opt.add_state(
    State(name="C", kT=6.2, traj_file=cg_traj_files[2], alpha=0.30)
)

## Add Pairs

In [7]:
pair0 = Pair(type1="E", type2="E")
pair1 = Pair(type1="K", type2="K")
pair2 = Pair(type1="E", type2="K")
for pair in [pair0, pair1, pair2]:
    pair.set_table_potential(epsilon=1, sigma=1, r_min=1e-4, r_max=4.0, n_points=101)
    opt.add_pair(pair)

## Add Bonds

In [8]:
bond0 = Bond(type1="E", type2="K")
bond1 = Bond(type1="K", type2="K")

bond0.set_harmonic(k=500, l0=2.7)
bond1.set_harmonic(k=450, l0=2.8)

opt.add_bond(bond0)
opt.add_bond(bond1)

## Add Angles

In [9]:
angle0 = Angle(type1="E", type2="K", type3="K")
angle1 = Angle(type1="K", type2="E", type3="K")

angle0.set_harmonic(k=50, theta0=2.4)
angle1.set_harmonic(k=50, theta0=2.4)

opt.add_angle(angle0)
opt.add_angle(angle1)

In [12]:
dir(angle0)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_state',
 '_compute_current_distribution',
 '_get_state_distribution',
 '_potential_file',
 '_save_current_distribution',
 '_states',
 '_update_potential',
 'angle_entry',
 'angle_init',
 'angle_type',
 'name',
 'potential',
 'previous_potential',
 'set_harmonic',
 'set_quadratic',
 'type1',
 'type2',
 'type3',
 'update_potential_file']

## Run Pair Optimization Iterations

In [10]:
opt.optimize_pairs(n_iterations=1)

-------- Iteration 0 --------
Running state A on GPU
Launched HOOMD in /home/chris/cme/forks/msibi/msibi/tutorials/peek/states/A_4.1
Finished in /home/chris/cme/forks/msibi/msibi/tutorials/peek/states/A_4.1.
Running state B on GPU
Launched HOOMD in /home/chris/cme/forks/msibi/msibi/tutorials/peek/states/B_5.5
Finished in /home/chris/cme/forks/msibi/msibi/tutorials/peek/states/B_5.5.
Running state C on GPU
Launched HOOMD in /home/chris/cme/forks/msibi/msibi/tutorials/peek/states/C_6.2
Finished in /home/chris/cme/forks/msibi/msibi/tutorials/peek/states/C_6.2.
pair E-E, state A, iteration 0: 0.882525
pair E-E, state B, iteration 0: 0.896128
pair E-E, state C, iteration 0: 0.881490
pair K-K, state A, iteration 0: 0.893351
pair K-K, state B, iteration 0: 0.905159


/home/chris/cme/forks/msibi/msibi/pair.py:306: RuntimeWarning: divide by zero encountered in true_divide
  kT * alpha * np.log(current_rdf[:,1] / target_rdf[:,1]) / N
/home/chris/cme/forks/msibi/msibi/pair.py:306: RuntimeWarning: invalid value encountered in true_divide
  kT * alpha * np.log(current_rdf[:,1] / target_rdf[:,1]) / N


pair K-K, state C, iteration 0: 0.896543
pair E-K, state A, iteration 0: 0.889500
pair E-K, state B, iteration 0: 0.900425
pair E-K, state C, iteration 0: 0.891097


/home/chris/cme/forks/msibi/msibi/pair.py:306: RuntimeWarning: divide by zero encountered in log
  kT * alpha * np.log(current_rdf[:,1] / target_rdf[:,1]) / N


In [11]:
for pair in [pair0, pair1, pair2]:
    print(pair._potential_file)
    print(pair.pair_init)
    print(pair.pair_entry)

/home/chris/cme/forks/msibi/msibi/tutorials/peek/potentials/pair_pot.E-E.txt
table=hoomd.md.pair.table(width=101,nlist=nl)
table.set_from_file('E', 'E', filename='/home/chris/cme/forks/msibi/msibi/tutorials/peek/potentials/pair_pot.E-E.txt')
/home/chris/cme/forks/msibi/msibi/tutorials/peek/potentials/pair_pot.K-K.txt
table=hoomd.md.pair.table(width=101,nlist=nl)
table.set_from_file('K', 'K', filename='/home/chris/cme/forks/msibi/msibi/tutorials/peek/potentials/pair_pot.K-K.txt')
/home/chris/cme/forks/msibi/msibi/tutorials/peek/potentials/pair_pot.E-K.txt
table=hoomd.md.pair.table(width=101,nlist=nl)
table.set_from_file('E', 'K', filename='/home/chris/cme/forks/msibi/msibi/tutorials/peek/potentials/pair_pot.E-K.txt')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline
for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    
plt.title("PM Pair")
#plt.legend()
plt.show()

for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-P.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("PP Pair")
#plt.legend()
plt.show()

for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.M-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("MM Pair")
#plt.legend()
plt.show()

In [ ]:
%matplotlib inline
for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    
plt.title("PM Pair")
#plt.legend()
plt.show()

for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-P.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("PP Pair")
#plt.legend()
plt.show()

for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.M-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("MM Pair")
#plt.legend()
plt.show()